In [171]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

from six.moves import urllib

import pandas as pd
import tensorflow as tf

In [201]:
# Read in data
import pandas as pd
import scipy.io
variable = pd.read_csv('/Users/posnerlab/Desktop/Project 2/BrainCNN/neonatalinfantbrainstudy/Variables.txt', sep="\t")

infant = pd.read_csv('/Users/posnerlab/Desktop/Project 2/BrainCNN/neonatalinfantbrainstudy/infant.csv', header=None)
infant_white = pd.read_csv('/Users/posnerlab/Desktop/Project 2/BrainCNN/neonatalinfantbrainstudy/infant_white.csv', header=None)
infant = infant.drop(0, 1)
infant = infant.drop(8099, 1)
infant_white = infant_white.drop(0, 1)
infant_white = infant_white.drop(8099, 1)
data = pd.DataFrame(pd.concat([infant, variable['SSRI']], axis=1))
data_white = pd.concat([infant_white, variable['SSRI']], axis=1)

# Rename edge names
new_cols = dict(zip(data.columns[0:data.shape[1]-1], ['edge_' + str(x) for x in data.columns[0:data.shape[1]-1]]))
data.rename(columns= new_cols, inplace=True)
data_white.rename(columns= new_cols, inplace=True)
data.to_csv('data.csv')
data_white.to_csv('data_white.csv')
data

,edge_1,edge_2,edge_3,edge_4,edge_5,edge_6,edge_7,edge_8,edge_9,edge_10,...,edge_8090,edge_8091,edge_8092,edge_8093,edge_8094,edge_8095,edge_8096,edge_8097,edge_8098,SSRI
0,4795,6541,1942,2365,2044,7596,1181,803,1301,2864,...,293,601,541,746,603,11937,1269,3694,2591,0
1,4265,5010,2839,1844,1857,6875,2165,1132,1640,3077,...,205,748,407,650,300,15548,1767,3644,2852,0
2,4887,4858,1472,1349,1446,4551,1532,773,1207,2357,...,104,273,306,450,243,12996,1953,3142,3085,0
3,4914,4679,2965,1668,2219,6755,2097,1071,1609,3491,...,128,687,338,456,311,12487,2078,5100,4112,0
4,5257,3946,3984,1556,2123,5717,2714,1387,1301,4669,...,213,548,192,726,274,18218,1632,3584,3483,0
5,5991,5225,3566,2944,3431,6900,1771,1501,2185,3410,...,80,221,272,238,378,9325,1123,2631,2016,1
6,5227,5827,4043,1910,2246,4844,3703,891,1359,1921,...,315,930,157,262,383,15861,1510,4662,3396,0
7,4925,6014,3686,2012,1884,5759,2222,1063,1057,3719,...,169,782,417,820,520,15467,1695,5840,3143,0
8,4860,4128,3415,1462,2213,5276,2934,983,1133,2925,...,55,950,437,474,1012,16144,1359,3706,3873,0
9,4683,4692,3075,1723,1557,6465,1979,1183,848,2008,...,330,616,259,490,334,14113,1050,3577,3271,0


In [202]:
# Split train and test data
from sklearn.model_selection import train_test_split
train_infant, test_infant = train_test_split(data, test_size = 0.2)
train_infant_white, test_infant_white = train_test_split(data_white, test_size = 0.2)
train_infant.to_csv('train_infant.csv')
test_infant.to_csv('test_infant.csv')
train_infant_white.to_csv('train_infant_white.csv')
test_infant_white.to_csv('test_infant_white.csv')

In [203]:
# Continuous base columns.
edge = []
for i in range(data.shape[1]-2):
    edge.append(tf.contrib.layers.real_valued_column(data.columns[i]))

In [204]:
# Deep columns
deep_columns = []
for i in range(data.shape[1]-2):
    deep_columns.append(edge[i])
# Wide columns
wide_columns = []

In [214]:
# Combine deep and wide columns
import tempfile
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.DNNLinearCombinedClassifier(
    model_dir=model_dir,
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[50, 50],
    n_classes=2)

Instructions for updating:
Please set fix_global_step_increment_bug=True and update training steps in your pipeline. See pydoc for details.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x129838590>, '_model_dir': '/var/folders/z1/n4w0rk993zl8406dtsghskvr0000gn/T/tmpFyEEk1', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}


In [215]:
import pandas as pd
import urllib
COLUMNS = list(data)
LABEL_COLUMN = 'label'
CONTINUOUS_COLUMNS = list(data[0:data.shape[1]-2])
CATEGORICAL_COLUMNS = []

train_file = tempfile.NamedTemporaryFile()
test_file = tempfile.NamedTemporaryFile()
urllib.urlretrieve("/Users/posnerlab/Desktop/BingHan/train_infant.csv", train_file.name)
urllib.urlretrieve("/Users/posnerlab/Desktop/BingHan/test_infant.csv", test_file.name)

('/var/folders/z1/n4w0rk993zl8406dtsghskvr0000gn/T/tmpX7dQGT',
 <mimetools.Message instance at 0x158b3a4d0>)

In [217]:
df_train = pd.read_csv(train_file, names=COLUMNS)
df_test = pd.read_csv(test_file, names=COLUMNS, skiprows=1)
df_train[LABEL_COLUMN] = (df_train['SSRI'].apply(lambda x: '1' in x)).astype(int)
df_test[LABEL_COLUMN] = (df_test['SSRI'].apply(lambda x: '1' in x)).astype(int)

df_train
#for col in df_train:
#    df_test[col] = df_test[col].astype(int)
#for col in df_test:
#    df_test[col] = df_test[col].astype(int)



,edge_1,edge_2,edge_3,edge_4,edge_5,edge_6,edge_7,edge_8,edge_9,edge_10,...,edge_8091,edge_8092,edge_8093,edge_8094,edge_8095,edge_8096,edge_8097,edge_8098,SSRI,label


In [218]:
df_train_1 = df_train.drop('SSRI', 1)
df_train_2 = df_train_1.drop('label', 1)

for col in df_train_2[0:8097]:
    df_train_2[col] = df_train_2[col].astype(int)

df_train_3 = pd.concat([df_train_2, df_train['SSRI']], axis=1)
df_train = pd.concat([df_train_3, df_train['label']], axis=1)

df_test_1 = df_test.drop('SSRI', 1)
df_test_2 = df_test_1.drop('label', 1)

for col in df_test_2[0:8097]:
    df_test_2[col] = df_test_2[col].astype(int)

df_test_3 = pd.concat([df_test_2, df_test['SSRI']], axis=1)
df_test = pd.concat([df_test_3, df_test['label']], axis=1)
df_test.dtypes

edge_1        int64
edge_2        int64
edge_3        int64
edge_4        int64
edge_5        int64
edge_6        int64
edge_7        int64
edge_8        int64
edge_9        int64
edge_10       int64
edge_11       int64
edge_12       int64
edge_13       int64
edge_14       int64
edge_15       int64
edge_16       int64
edge_17       int64
edge_18       int64
edge_19       int64
edge_20       int64
edge_21       int64
edge_22       int64
edge_23       int64
edge_24       int64
edge_25       int64
edge_26       int64
edge_27       int64
edge_28       int64
edge_29       int64
edge_30       int64
              ...  
edge_8071     int64
edge_8072     int64
edge_8073     int64
edge_8074     int64
edge_8075     int64
edge_8076     int64
edge_8077     int64
edge_8078     int64
edge_8079     int64
edge_8080     int64
edge_8081     int64
edge_8082     int64
edge_8083     int64
edge_8084     int64
edge_8085     int64
edge_8086     int64
edge_8087     int64
edge_8088     int64
edge_8089     int64


In [219]:
def input_fn(df):
    """Input builder function."""
    # Creates a dictionary mapping from each continuous feature column name (k) to
    # the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values, shape=[df[k].size, 1]) for k in CONTINUOUS_COLUMNS}
    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored in a tf.SparseTensor.
    categorical_cols = {k: tf.SparseTensor(
            indices=[[i, 0] for i in range(df[k].size)],
            values=df[k].values,
            dense_shape=[df[k].size, 1])
                   for k in CATEGORICAL_COLUMNS}
    # Merges the two dictionaries into one.
    feature_cols = dict(continuous_cols.items() + categorical_cols.items())
    #feature_cols.update(categorical_cols)
    # Converts the label column into a constant Tensor.
    label = tf.constant(df[LABEL_COLUMN].values)
    # Returns the feature columns and the label.
    return feature_cols, label

In [220]:
def train_input_fn():
    return input_fn(df_train)

def eval_input_fn():
    return input_fn(df_test)

In [221]:
m.fit(input_fn=train_input_fn, steps=200)
results = m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print("%s: %s" % (key, results[key]))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/z1/n4w0rk993zl8406dtsghskvr0000gn/T/tmpFyEEk1/model.ckpt.
ERROR:tensorflow:Model diverged with loss = NaN.


NanLossDuringTrainingError: NaN loss during training.